# Load Packages

In [11]:
# Importing some libraries
import scrapy
from scrapy.crawler import CrawlerProcess
from collections import deque
import json
import numpy as np

# Crawling

i explained everything with comment and i think code is clear.

In [2]:
# This is a template to help you implement your crawler. Feel free to modify it.
class WebSpider(scrapy.Spider):
    name = 'web spider'
    queued_urls = set()
    count = 0
    
    def __init__(self, start_urls, fetch_limit):
        self.start_urls = start_urls
        self.fetch_limit = fetch_limit
        self.queued_urls = set(start_urls)
        self.count = 0

    def parse(self, response):
        
        # check crawl fetch limitation
        if (self.count < self.fetch_limit):
            self.count += 1
            
            yield {
                'id': response.css('link[rel=canonical]::attr(href)').get().split('/')[-1],
                # Extract title from title tag in html and exclued ("| semanticscholar") from last of it and ("[PDF]") from first of it
                'title': response.css('title::text').get()[: -19].replace("[PDF] ", ""),
                # extrac references from section references and hyperlink tag (a) and just return paper id
                'references': response.xpath('//div[@id="references"]//a/@data-heap-paper-id').getall()
                }
        else:
            return
                
        
        
        
        
        
        # check crawl fetch limitation
        if(self.count >= self.fetch_limit):
            return
        
        # address base for carwling
        base = "https://www.semanticscholar.org"
        
        # get url of references for crawling
        article_urls = response.xpath('//div[@id="references"]//a[contains(@data-heap-id, "citation_title")]/@href').getall()
        
        # crawl in references
        for i in range(len(article_urls)):
            
            # create complete url
            url = base + article_urls[i]
            
            # check if we did not crawl it before
            if (url not in self.queued_urls):
                
                # add it to crawled url
                self.queued_urls.add(url)
                
                # start crawling
                yield scrapy.Request(url=url, callback=self.parse)
                
def crawl_all(start_urls, fetch_limit = 2000):
    class WebSpiderTemp(WebSpider):
        def __init__(self):
            super().__init__(start_urls, fetch_limit)

    process = CrawlerProcess(settings={
        "FEEDS": {
            "all.json": {"format": "json"},
        },
    })
    process.crawl(WebSpiderTemp)
    process.start()

# Crawl 1000 articles

In [3]:
start_urls = [
    'https://www.semanticscholar.org/paper/Inductive-Representation-Learning-on-Large-Graphs-Hamilton-Ying/6b7d6e6416343b2a122f8416e69059ce919026ef',
    # Add more papers if you want.
]   
crawl_all(start_urls, 1000)

2021-01-04 18:58:52 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2021-01-04 18:58:52 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.9.2, Platform Windows-10-10.0.19041-SP0
2021-01-04 18:58:52 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-01-04 18:58:52 [scrapy.crawler] INFO: Overridden settings:
{}
2021-01-04 18:58:52 [scrapy.extensions.telnet] INFO: Telnet Password: 815e6175d460de35
2021-01-04 18:58:52 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-01-04 18:58:53 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloade

2021-01-04 18:58:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Semi-Supervised-Classification-with-Graph-Networks-Kipf-Welling/36eff562f65125511b5dfab68ce7f7a943c27478> (referer: https://www.semanticscholar.org/paper/Inductive-Representation-Learning-on-Large-Graphs-Hamilton-Ying/6b7d6e6416343b2a122f8416e69059ce919026ef)
2021-01-04 18:58:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.semanticscholar.org/paper/Discriminative-Embeddings-of-Latent-Variable-Models-Dai-Dai/322cf9bcde458a45eaeca989a1eec92f7c6db984>
{'id': '322cf9bcde458a45eaeca989a1eec92f7c6db984', 'title': 'Discriminative Embeddings of Latent Variable Models for Structured Data', 'references': ['ffaa313b8da3695627cd9915ca46b8bed24a9f4a', 'e45c2420e6dc59ba6d357fb0c996ebf43c861560', '633664376ffadc1396d86e476925121ac3d4cf86', 'e49ff72d420c8d72e62a9353e3abc053445e59bd', 'd98d0d1900b13b87aa4ffd6b69c046beb63f0434', 'c66758c1029a463489f26aeb3955f333b37f727a', '1e20f9de45d26

2021-01-04 18:59:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Understanding-the-difficulty-of-training-deep-Glorot-Bengio/b71ac1e9fb49420d13e084ac67254a0bbd40f83f> (referer: https://www.semanticscholar.org/paper/Variational-Graph-Auto-Encoders-Kipf-Welling/54906484f42e871f7c47bbfe784a358b1448231f)
2021-01-04 18:59:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Generalization-of-back-propagation-to-recurrent-Pineda/6602985bd326d9996c68627b56ed389e2c90fd08> (referer: https://www.semanticscholar.org/paper/A-new-model-for-learning-in-graph-domains-Gori-Monfardini/9ca9f28676ad788d04ba24a51141a9a0a0df4d67)
2021-01-04 18:59:00 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.semanticscholar.org/paper/Understanding-the-difficulty-of-training-deep-Glorot-Bengio/b71ac1e9fb49420d13e084ac67254a0bbd40f83f>
{'id': 'b71ac1e9fb49420d13e084ac67254a0bbd40f83f', 'title': 'Understanding the difficulty of training

2021-01-04 18:59:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Laplacian-Eigenmaps-and-Spectral-Techniques-for-and-Belkin-Niyogi/9d16c547d15a08091e68c86a99731b14366e3f0d> (referer: https://www.semanticscholar.org/paper/LINE%3A-Large-scale-Information-Network-Embedding-Tang-Qu/0834e74304b547c9354b6d7da6fa78ef47a48fa8)
2021-01-04 18:59:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.semanticscholar.org/paper/Laplacian-Eigenmaps-and-Spectral-Techniques-for-and-Belkin-Niyogi/9d16c547d15a08091e68c86a99731b14366e3f0d>
{'id': '9d16c547d15a08091e68c86a99731b14366e3f0d', 'title': 'Laplacian Eigenmaps and Spectral Techniques for Embedding and Clustering', 'references': ['afcd6da7637ddeef6715109aca248da7a24b1c65', '95d6ff6279fa0f92df6fae0e6bd4c259acfc8f09', '3537fcd0ff99a3b3cb3d279012df826358420556', 'b94c7ff9532ab26c3aedbee3988ec4c7a237c173', '8465abc8b655b86f5009305ed32882bfece72475']}
2021-01-04 18:59:02 [scrapy.core.engine] DEBUG: Crawle

2021-01-04 18:59:03 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.semanticscholar.org/paper/Why-collective-inference-improves-relational-Jensen-Neville/54c5f8c99175cd22c4dd2c1a7221824daf99fec2>
{'id': '54c5f8c99175cd22c4dd2c1a7221824daf99fec2', 'title': 'Why collective inference improves relational classification', 'references': ['9cc36397e1fef5c922d64e88211a7e08ecc64759', '5519517f748473a0457f6e4ba86522df418eaee1', '645ad00985d65c7e846832f8e19917e0629b9b7e']}
2021-01-04 18:59:04 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.semanticscholar.org/paper/Characteristics-of-Random-Nets-of-Analog-Elements-Amarimber/3ab557ca114ad56e53910add3fdf4325cea700bb>
{'id': '3ab557ca114ad56e53910add3fdf4325cea700bb', 'title': 'Characteristics of Random Nets of Analog Neuron-Like Elements', 'references': ['32da988e196e4d6805cc71aaa90bcef554a5e454', '88a38a94345c726de22bf786261a539262650967', '2e3e96100b1365b235b4f639255a94810e139d51', '73a33c1f5131ec1558adae62a351592f0b403a34',

...

2021-01-04 19:07:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Recommending-and-evaluating-choices-in-a-virtual-of-Hill-Stead/c864175780adde19099108de66b4636f4c87d44c> (referer: https://www.semanticscholar.org/paper/Incremental-Singular-Value-Decomposition-Algorithms-Sarwar-Karypis/02ff37cd0059cf1af1ecfa62c32304c05ab3bf96)
2021-01-04 19:07:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/GroupLens%3A-an-open-architecture-for-collaborative-Resnick-Iacovou/64a717dc148b76070bbb6a3190a7e05bb9734400> (referer: https://www.semanticscholar.org/paper/Incremental-Singular-Value-Decomposition-Algorithms-Sarwar-Karypis/02ff37cd0059cf1af1ecfa62c32304c05ab3bf96)
2021-01-04 19:07:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/An-Algorithmic-Framework-for-Performing-Filtering-Herlocker-Konstan/cc8d824ad7ee47f2d2f56b80125810519ea7f51e> (referer: https://www.semanticscholar.org/pape

2021-01-04 19:07:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Deep-Gaussian-Processes-Damianou-Lawrence/b5f5ed292ce93404a0985754b2b531eca4e6cf50> (referer: https://www.semanticscholar.org/paper/Stochastic-Backpropagation-and-Approximate-in-Deep-Rezende-Mohamed/484ad17c926292fbe0d5211540832a8c8a8e958b)
2021-01-04 19:07:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/An-Examination-of-Experimental-Methodology-for-of-Gallagher-Eliassi-Rad/e16c2eea4ffedd2325c7d530e5ed581c6babd92a> (referer: https://www.semanticscholar.org/paper/Collective-Classification-in-Network-Data-Sen-Namata/c5f2f13778af201f486b0b3c4c8f6fcf36d4ca36)
2021-01-04 19:07:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Link-based-classification-Lu-Getoor/784220a5a2ad452282f8af006a6cb5715d54d0ed> (referer: https://www.semanticscholar.org/paper/Collective-Classification-in-Network-Data-Sen-Namata/c5f2f137

2021-01-04 19:07:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Neural-Network-for-Graphs%3A-A-Contextual-Approach-Micheli/ec2b2569b3a0d70a5b45d48b041dec9060d85eb7> (referer: https://www.semanticscholar.org/paper/Gated-Graph-Sequence-Neural-Networks-Li-Tarlow/492f57ee9ceb61fb5a47ad7aebfec1121887a175)
2021-01-04 19:07:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/The-Graph-Neural-Network-Model-Scarselli-Gori/3efd851140aa28e95221b55fcc5659eea97b172d> (referer: https://www.semanticscholar.org/paper/Gated-Graph-Sequence-Neural-Networks-Li-Tarlow/492f57ee9ceb61fb5a47ad7aebfec1121887a175)
2021-01-04 19:07:53 [scrapy.extensions.logstats] INFO: Crawled 2169 pages (at 180 pages/min), scraped 1000 items (at 0 items/min)
2021-01-04 19:07:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Semi-Supervised-Learning-with-Spectral-Graph-Shuman-Faraji/4a5a12f0ffaad71edd2d77a0fdd134f65

2021-01-04 19:08:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Deep-Convolutional-Networks-on-Graph-Structured-Henaff-Bruna/e49ff72d420c8d72e62a9353e3abc053445e59bd> (referer: https://www.semanticscholar.org/paper/Discriminative-Embeddings-of-Latent-Variable-Models-Dai-Dai/322cf9bcde458a45eaeca989a1eec92f7c6db984)
2021-01-04 19:08:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Learning-Deep-Representations-for-Graph-Clustering-Tian-Gao/df787a974fff59f557ed1ec620fc345568aec491> (referer: https://www.semanticscholar.org/paper/Revisiting-Semi-Supervised-Learning-with-Graph-Yang-Cohen/3d846cb01f6a975554035d2210b578ca61344b22)
2021-01-04 19:08:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.semanticscholar.org/paper/Exploiting-Generative-Models-in-Discriminative-Jaakkola-Haussler/e45c2420e6dc59ba6d357fb0c996ebf43c861560> (referer: https://www.semanticscholar.org/paper/Discriminative-Embeddings-o

2021-01-04 19:10:43 [scrapy.core.engine] INFO: Closing spider (finished)
2021-01-04 19:10:43 [scrapy.extensions.feedexport] INFO: Stored json feed (1000 items) in: all.json
2021-01-04 19:10:43 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/exception_count': 6,
 'downloader/exception_type_count/twisted.internet.error.TimeoutError': 6,
 'downloader/request_bytes': 1969312,
 'downloader/request_count': 2216,
 'downloader/request_method_count/GET': 2216,
 'downloader/response_bytes': 114710378,
 'downloader/response_count': 2210,
 'downloader/response_status_count/200': 2210,
 'elapsed_time_seconds': 710.488291,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 1, 4, 15, 40, 43, 832819),
 'item_scraped_count': 1000,
 'log_count/DEBUG': 3216,
 'log_count/INFO': 22,
 'request_depth_max': 43,
 'response_received_count': 2210,
 'retry/count': 6,
 'retry/reason_count/twisted.internet.error.TimeoutError': 6,
 'scheduler/dequeued': 2216,
 'scheduler/dequeued

## Load Json File

In [252]:
# Opening JSON file
with open('all.json') as f: 
    data = json.load(f)

# convert list of articles into dict with id of papers as a key
ready_data = {}
for article in data:
    ready_data[article['id']] = article
    

######################### important ########################
# because of fetch limit we can't crawl all references, so here i deleted each reference that hasn't been crawled

# paper ids
keys = list(ready_data.keys())

# iterate on each article to find and delete references that hasn't been crawled
for key in ready_data:
    
    # list of article references
    ref_list = ready_data[key]['references']
    list_h = ref_list[:]
    
    # iterate on references
    for ref in ref_list:
        
        # if this reference is not crawled ( is not in list of crawled paper ) then remove it
        if(ref not in keys):
            list_h.remove(ref)
    
    # save new list of references
    ready_data[key]['references'] = list_h
    # save out degree of each article
    ready_data[key]['out_degree'] = len(list_h)
    

# here we compute predecessor list of each article which are papers that have links to this paper
for key1 in keys:
    
    # list of predecessor
    pred_list = []
    
    # iterate on papers to find papers that has key1 (paper) in its reference list
    # ( we don't need to exclude id of current paper, because no article has a reference to itself)
    for key2 in keys:
        
        # if key1 is in references list key2, add key2 as key1's predecessor
        if(key1 in ready_data[key2]['references']):
            pred_list.append(key2)
    
    # add list of predecessor to each article
    ready_data[key1]['pred_list'] = pred_list
    

            
        
    

# PageRank with Predecessor List

In [253]:
# here we compute pagerank with predecessors list
def pagerank(M, a, iter_tol):
    
    # number of articles or nodes of graph
    n = len(M)
    
    # initial value of pageranks
    v = (1/n) * np.ones(n)
    
    # holding pagerank value to check convergence at end of each round
    v_old = np.empty_like(v)
    
    # list of articles id
    ids = list(M.keys())
    
    while(1):
        
        # clone or copy v into v_old
        v_old[:] = v 
        
        # iterate on each article
        for i in range(n):

            # find list of its predecessors
            pred_list = M[ids[i]]['pred_list']
            
            # new value of pagerank for this( i-th article ) article
            new_v = 0
            
            # iterate on list of predecessors
            for j in pred_list:
                
                # find index of id of thid predecessor int ids list
                ref_ind = ids.index(j)
                
                # find out degree of (j) predecessor
                degree = M[j]['out_degree']
                
                # add value that this predecessor adds to successor article
                new_v += (1 / degree) * v_old[ref_ind]
            
            # using taxation formula to compute new value of pagerank
            v[i] = (1 - a) * new_v +  (a / n)
            
        # using l1 norm to find convergence
        diff = sum(np.abs(v - v_old))
    
        # check for convergence
        if( diff < iter_tol ):
            break
            
    return v
    

In [254]:
# finding pagerank with a = 0.15
pagerank_list = pagerank(ready_data, 0.15, 0.00001)

# Adjacancy Matrix

In [255]:
# we just use this matrix for second implementation of pagerank 

# holding n*n array
A = np.zeros((len(ready_data), len(ready_data)))

# iterate on each article
for key1 in keys:
    
    # index of key1 in list of all ids
    ind1 = keys.index(key1)
    
    # iterate to find if key1 is predecessor of key2
    for key2 in keys:    
        
        # if key2 is predecessor of key1, fill value of its adjacency matrix with ( 1 / out_degree(key2) )
        if(key1 in ready_data[key2]['references']):
            
            ind2 = keys.index(key2)
            A[ind1][ind2] = ( 1 / ready_data[key2]['out_degree'])
            
            

# PageRank with Adjacancy Matrix

In [256]:
# matrix implementation of pageranl
def pagerank_mat(M, a, iter_tol):
    
    # number of articles or nodes of graph
    n = len(M)
    
    # initial value of pageranks
    v = (1/n) * np.ones(n)
    
    # fix probablity value of new surfer 
    new_surfer = (a/n) * np.ones(n)
    
    while(1):
        
        # nex round value of pagerank ( taxation formula )
        new_v = (1-a) * ( M @ v ) + new_surfer
        
        # using l1 norm to find convergence
        diff = sum(np.abs(new_v - v))
        
        # clone or copy new_v into v
        v[:] = new_v
        
        # check for convergence
        if( diff < iter_tol ):
            break
    
    return v

In [257]:
# finding pagerank with a = 0.15
pagerank_list = pagerank_mat(A, 0.15, 0.000001)

# Search with Pagerank

In [258]:
# finding list of articles that have a specific word in their title
def search_pr(word, pagerank_list, M):
    
    # convert word to lowercase ( avoid case sensetivity )
    word = word.lower()
    
    # list of result
    result_list = []
    
    # list of articles id
    ids = list(M.keys())
    
    # iterate on articles
    for ind in range(len(ids)):
        
        # extract their title
        title_article = M[ids[ind]]['title']
        
        # convert title to lower case and check if it includes word
        if( word in  title_article.lower() ):
            
            # hold result as a dict with title and pagerank
            dict_search = {}
            dict_search['title'] = title_article
            dict_search['pagerank'] = pagerank_list[ind]
            
            result_list.append(dict_search)
            
    # sort result dict with pagerank value    
    sorted_list = sorted(result_list, key = lambda i: i['pagerank'], reverse = True)
    
    # print result
    for res in sorted_list:
        print(res)
    
    


In [259]:
search_pr('optimization', pagerank_list, ready_data)

{'title': 'Geometric Algorithms and Combinatorial Optimization', 'pagerank': 0.00036666237513107716}
{'title': 'Stochastic quasigradient methods and their application to system optimization', 'pagerank': 0.0003200981989787573}
{'title': 'Simulation-based optimization of Markov decision processes', 'pagerank': 0.00027256559560850116}
{'title': 'Introductory Lectures on Convex Optimization - A Basic Course', 'pagerank': 0.00025515273395356196}
{'title': 'On convergence rates of subgradient optimization methods', 'pagerank': 0.00025515273395356196}
{'title': 'Incremental subgradient methods for nondifferentiable optimization', 'pagerank': 0.00024741819753779294}
{'title': 'An improved subgradient method for constrained nondifferentiable optimization', 'pagerank': 0.0002149454650251953}
{'title': 'The optimization of a quadratic function subject to linear constraints', 'pagerank': 0.0002128303537631886}
{'title': 'Adam: A Method for Stochastic Optimization', 'pagerank': 0.00016383374999999

# HITS

In [260]:
# computing HITS with list algorithm that was mentioned at first of Question 2 
def hits(M, num_iter, iter_tol):
    
    
    # list of articles id
    ids = list(M.keys())
    
    # initial values of h and a with all 1
    h = dict.fromkeys(ids, 1)
    a = dict.fromkeys(ids, 1)
    
    # number of rounds
    for i in range(num_iter):  
        
        # holding last h ( copy without reference )
        hlast = dict(h)
    
        # computing new h with summation on its authorities
        for key in h:
            
            # new value of h
            new_h = 0
            
            # iterate on successors of key ( becaues successors are authorities)
            for succ_key in M[key]['references']:
                
                # add value of authirity to new_h
                new_h += a[succ_key]
            
            # update hubs value for key
            h[key] = new_h   
        
        # normalize h as question mentioned with maximum value of h
        inv_max_h = 1 / max(h.values())
        for key in h:
            h[key] *= inv_max_h
        
        # computing new a with summation on its hubs
        for key in a:
            
            # new value of a
            new_a = 0
            
            # iterate on predecessors of key ( becaues predecessors are hubs)
            for pred_key in M[key]['pred_list']:
                
                # add value of hubs to new_a
                new_a += h[pred_key]
            
            # update authority value for key
            a[key] = new_a
               
        
        # normalize a as question mentioned with maximum value of a
        inv_max_a = 1 / max(a.values())
        for key in a:
            a[key] *= inv_max_a
        
        # using l1 norm to find convergence
        # convert dicts value to numpy array
        hlast_val = np.array(list(hlast.values()))
        h_val = np.array(list(h.values()))
        
        # find l1 norm
        diff = sum(np.abs(hlast_val - h_val))
        
        # check for convergence
        if( diff < iter_tol ):
            break
        
    
    # return value of hubs and authorities and convergence iteration number
    return h, a, i+1

# Printing Top 20 Hubs and Authorities

In [261]:
h_vector, a_vector, conv_iter = hits(ready_data, 1000, 0.0005)

print("Number of iteration for convergence = " + str(conv_iter))

print("\n##################### Top 20 hubs ######################")
h_sorted = sorted(h_vector.items(), key = lambda x: x[1], reverse=True)[:20]

for i in h_sorted:
    res_dict = {}
    
    res_dict['title'] = ready_data[i[0]]['title']
    res_dict['h'] = i[1]
    
    print(res_dict)
    
print("\n##################### Top 20 authorities ######################")
a_sorted = sorted(a_vector.items(), key = lambda x: x[1], reverse=True)[:20]

for i in a_sorted:
    res_dict = {}
    
    res_dict['title'] = ready_data[i[0]]['title']
    res_dict['a'] = i[1]
    
    print(res_dict)    

Number of iteration for convergence = 796

##################### Top 20 hubs ######################
{'title': 'Neue Beiträge zur Kenntniss der Zelle', 'h': 1.0}
{'title': 'Beiträge zur Kenntniss der Zelle und ihrer Lebenserscheinungen', 'h': 0.8785916809774557}
{'title': 'Ueber die Befruchtung bei Arion empiricorum', 'h': 0.8106981411187614}
{'title': 'Beiträge zur Kenntniss der Zelle und ihrer Theilungserscheinungen', 'h': 0.7595400364640432}
{'title': 'Beiträge zur Kenntniss der Zelle und Ihrer Lebenserscheinungen', 'h': 0.7099811148686526}
{'title': 'Ueber Karyokinese und ihre Beziehungen zu den Befruchtungsvorgängen', 'h': 0.6532935886370592}
{'title': 'Untersuchungen über die doppelte Form der Samenkörper von Paludina vivipara', 'h': 0.6281604274845233}
{'title': 'Das menschliche Ei', 'h': 0.5930377752933687}
{'title': 'Zur bildung der Geschlechtsprodukte bei den Pulmonaten', 'h': 0.571047200597315}
{'title': 'Beiträge zur Kenntniss der Zelle und ihrer Theilung', 'h': 0.4132740149

# Shortest Path Between to Articles

In [262]:
# Implementation of BFS 

def bfs(first_point, second_point, M):
    
    # number of vertex
    n = len(M)
    
    # id of articles
    ids = list(M.keys())
    
    # queue for holding vertexes which we want to add their neighbors
    queue = deque([first_point])
    
    # list for checking if we check vertex before
    checked_vertex = n*[0]
    checked_vertex[ids.index(first_point)] = 1
    
    # list for holding distance of each vertex to start point
    distance_list = n*[np.inf]
    distance_list[ids.index(first_point)] = 0
    
    # list for holding predecessor of vertexes in shortest path
    predecessor_list = n*[0]
    
    # continue till queue is empty
    while( len(queue) != 0 ):
        
        # vertex that we should add its neighbor ( we pop it because it should be removed after we check its neighbors)
        vertex = queue.pop()
        
        # iterate on its neighbors
        for neighbor in M[vertex]['references']:
            
            # check if not we checked this vertex before
            if( checked_vertex[ids.index(neighbor)] == 0 ):
                
                # value of its distance is distance of its pred + 1
                distance_list[ids.index(neighbor)] = distance_list[ids.index(vertex)] + 1
                
                # its predecessor is vertex
                predecessor_list[ids.index(neighbor)] = vertex
                
                # check this neighbor as visited so we didn't check it later
                checked_vertex[ids.index(neighbor)] = 1
                
                # add this to queue so we check its neighbor later
                queue.append(neighbor)
                
                # if we get to second point our work is done
                if( neighbor == second_point ):
                    
                    return True, distance_list, predecessor_list
                
    
    # if not found any path
    return False, distance_list, predecessor_list

# Test BFS

In [263]:
# id of articles
ids = list(ready_data.keys())

# id of first article
first_id = "6b7d6e6416343b2a122f8416e69059ce919026ef"

# id of second article
second_id = "c5f2f13778af201f486b0b3c4c8f6fcf36d4ca36"

# finding shortest path between two article
find_flag, distances, predecessors = bfs(first_id, second_id, ready_data)

if( find_flag ):
    print("Path details:")
    print("Distance is = " + str(distances[ids.index(second_id)]))
    
    print("path :")
    vertex_point = second_id
    path = second_id
    for i in range(distances[ids.index(second_id)]):
        
        vertex_point = predecessors[ids.index(vertex_point)]
        
        path = path + " <---- " + vertex_point
        
    print(path)    
        
    
else:
    print("no path was found!!!")

Path details:
Distance is = 2
path :
c5f2f13778af201f486b0b3c4c8f6fcf36d4ca36 <---- 54906484f42e871f7c47bbfe784a358b1448231f <---- 6b7d6e6416343b2a122f8416e69059ce919026ef
